In [40]:
# %pip install huggingface_hub
# %pip install llama_index

  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 27.5 MB/s eta 0:00:0000:0100:01
Using cached Deprecated-1.2.14-py2.py3-none-any.whl (9.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 10.0 MB/s eta 0:00:00
Using cached joblib-1.3.2-py3-none-any.whl (302 kB)
Note: you may need to restart the kernel to use updated packages.


In [23]:

from dataclasses import dataclass, asdict

from huggingface_hub import hf_hub_download, snapshot_download
from transformers import AutoTokenizer

from gdoc_sync.utils import upsert_folder
from nbdev.showdoc import patch_to


In [24]:
@dataclass
class ModelPath:
    install_path: str
    cache_path : str = None
    embedding_path : str = None

    def __post_init__(self):
        self.cache_path = self.cache_path or f"{self.install_path}/cache/"
        self.embedding_path = self.embedding_path or f"{self.install_path}/embedding/"



In [25]:
asdict(ModelPath(install_path = "../../models"))

{'install_path': '../../models',
 'cache_path': '../../models/cache/',
 'embedding_path': '../../models/embedding/'}

In [26]:


@dataclass
class ModelConfig:
      
  install_path : str

  prompt_style : str= "llama2"
  llm_hf_repo_id  : str= "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
  llm_tokenizer_name : str = 'mistralai/Mistral-7B-Instruct-v0.1'
  llm_hf_model_file :str = "mistral-7b-instruct-v0.2.Q4_K_M.gguf"
  embedding_hf_model_name :str= "BAAI/bge-small-en-v1.5"


  model_paths : ModelPath = None

  def __post_init__(self):
    self.model_paths = ModelPath(install_path= self.install_path )



In [27]:

@patch_to(ModelConfig)
def setup_model_folders(self, debug_prn: bool = False):
    
    [upsert_folder(folder_path, debug_prn = debug_prn) for folder_path in asdict(self.model_paths).values()]
    
    return True


In [28]:
config = ModelConfig(install_path='../../models/')
config.setup_model_folders( debug_prn = True)

{'upsert_folder': '/mnt/c/users/jaewi/GitHub/gdoc_sync/nbs/projects/models', 'is_exist': True}
{'upsert_folder': '/mnt/c/users/jaewi/GitHub/gdoc_sync/nbs/projects/models/cache', 'is_exist': True}
{'upsert_folder': '/mnt/c/users/jaewi/GitHub/gdoc_sync/nbs/projects/models/embedding', 'is_exist': True}


True

In [29]:

@patch_to(ModelConfig)
def download_embeddings_model(self):
    print(f"Downloading embedding {self.embedding_hf_model_name}")
    
    snapshot_download(
        repo_id=self.embedding_hf_model_name,
        cache_dir=self.model_paths.cache_path,
        local_dir=self.model_paths.embedding_path,
    )
    print(f"Embedding model downloaded")


In [30]:
config.download_embeddings_model()

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Embedding model downloaded


In [31]:

@patch_to(ModelConfig)
def download_llm_model(self, resume_download : bool = True):
    """Download LLM and create a symlink to the model file"""
    print(f"Downloading LLM {self.llm_hf_model_file}")
    
    hf_hub_download(
        repo_id=self.llm_hf_repo_id,
        filename=self.llm_hf_model_file,
        cache_dir=self.model_paths.cache_path,
        local_dir=self.model_paths.install_path,
        resume_download=resume_download,
    )
    print("LLM model downloaded!")

In [32]:
config.download_llm_model()

LLM model downloaded!


In [33]:

@patch_to(ModelConfig)
def download_tokenizer(self):
    print(f"Downloading tokenizer {self.llm_tokenizer_name}")
    
    AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=self.llm_tokenizer_name,
        cache_dir=self.model_paths.cache_path
    )
    print("Tokenizer downloaded!")

In [34]:
config.download_tokenizer()

Tokenizer downloaded!


In [37]:

@patch_to(ModelConfig)
def setup(self):
    self.download_embeddings_model()
    self.download_llm_model()
    self.download_tokenizer()

In [38]:
config.setup()

Fetching 13 files:   0%|          | 0/13 [00:00<?, ?it/s]

Embedding model downloaded
LLM model downloaded!
Tokenizer downloaded!


In [41]:
from llama_index.embeddings.base import BaseEmbedding

class EmbeddingComponent:
    embedding_model: BaseEmbedding
    
    def __init__(self, model_config: ModelConfig) -> None:

        from llama_index.embeddings import HuggingFaceEmbedding

        self.embedding_model = HuggingFaceEmbedding(
            model_name=model_config.embedding_hf_model_name,
            cache_folder=str(model_config.model_paths.cache_path),
        )